In [1]:
import numpy as np
from BPnumba.NumAG import InstancePob,TrainAG
from BPmodule.BPGen  import PROBLEM2,PROBLEM3,PROBLEM4,PROBLEM5
from BPnumba.NumFun import CreatePoblation
from numba.typed import List as NumbaList
from numba.typed import Dict
from numba import njit,prange
from typing import List
import time

In [ ]:
P2A1 = PROBLEM2[0]
P2A2 = PROBLEM2[1]
P3A1 = PROBLEM3[0]
P3A2 = PROBLEM3[1]
P4A1 = PROBLEM4[0]
P4A2 = PROBLEM4[1]
P5A1 = PROBLEM5[0]
P5A2 = PROBLEM5[1]



In [ ]:
contenedor = np.asanyarray(P2A1[0],dtype=np.int64)
datos =np.asanyarray(P2A1[1],dtype=np.int64)
initial = CreatePoblation(1,datos)
start_time = time.time()
Pob =InstancePob(NumbaList(initial),datos,contenedor)
TrainAG(2,NumbaList(Pob),0.85,0.75,0.2,NumbaList(datos),NumbaList(contenedor))
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
@njit(parallel=True)
def TestAlg(maxTest, maxItr,pop,datos, contenedor):
    initial = CreatePoblation(pop,datos)
    for _ in prange(maxTest):
        Pob = InstancePob(NumbaList(initial.copy()),datos,contenedor)
        best =TrainAG(maxItr,NumbaList(Pob),0.004,NumbaList(datos),NumbaList(contenedor))
        print(best.fi,len(datos)-best.load)

In [12]:
TestAlg(100,500,50,P5A1[1],P5A1[0])

In [ ]:
TestAlg(100,500,50,P4A1[1],P4A1[0])

In [ ]:
TestAlg(100,500,50,P4A2[1],P4A2[0])